## Partial Least Squares Regression (PLS)

### Table of Contents

1.  [Import Data](#import_data)
    a.
2.  [Grid Search](#grid_search)
3.  [Base Model Training](#base_model_training)


### Import Data <a name="import_data"></a>

Import the training and test split datasets generated from "preprocessing.ipynb".

In [1]:
import pandas as pd

data_file_path = '../../../data'

train_df = pd.read_csv(f'{data_file_path}/train_data.csv', index_col=0)
test_df = pd.read_csv(f'{data_file_path}/test_data.csv', index_col=0)

# drop 's' column

train_df = train_df.copy().drop(columns='s')
test_df = test_df.copy().drop(columns='s')

### Grid Search <a name="grid_search"></a>

In [2]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import make_scorer, r2_score
import pandas as pd

# Suppose you already have train_df and test_df
# with 'y' as your target column

X_train = train_df.drop(columns='y')
y_train = train_df['y']

# Define PLS model (no n_components specified yet)
pls = PLSRegression(scale=True, copy=True)

# Define the parameter grid for n_components
param_grid = {
    'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'max_iter': [500, 1000, 5000, 10000],
    'tol': [1e-9, 1e-10, 1e-11, 1e-12, 1e-13]
}

# Define a cross-validation strategy
cv = KFold(n_splits=10, shuffle=True)

# Optionally define a scoring metric (or just pass a string to 'scoring')
# scoring = make_scorer(r2_score)

grid_search = GridSearchCV(
    estimator=pls,
    param_grid=param_grid,
    scoring='neg_mean_squared_error', # 'r2'
    cv=cv,
    n_jobs=1,
    verbose=1
)

# Fit the grid search on training data
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=PLSRegression(), n_jobs=1,
             param_grid={'max_iter': [500, 1000, 5000, 10000],
                         'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                          12],
                         'tol': [1e-09, 1e-10, 1e-11, 1e-12, 1e-13]},
             scoring='neg_mean_squared_error', verbose=1)


#### Base Model Training <a name="base_model_training"></a>


In [10]:
grid_search.best_estimator_['max_iter']

StackingClassifier(estimators=[('pls',
                                PLSRegression(n_components=3, tol=1e-09))],
                   final_estimator=LogisticRegression())

In [9]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimators = [('pls', PLSRegression(n_components=grid_search.best_estimator_['n_components'],
                    scale=True,
                    max_iter=grid_search.best_estimator_['max_iter'],
                    tol=grid_search.best_estimator_['tol'],
                    copy=True))]

base_model = StackingClassifier(estimators=estimators,
                                final_estimator=LogisticRegression(penalty='l2'))

base_model.fit(X=train_df.copy().drop(columns='y'), y=train_df['y'])

StackingClassifier(estimators=[('pls',
                                PLSRegression(n_components=3, tol=1e-09))],
                   final_estimator=LogisticRegression())

#### Cross Validation

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(base_model,
                         X=train_df.copy().drop(columns='y'),
                         y=train_df['y'], cv=5, scoring='accuracy')
scores

0.72

#### Base Model Evaluation

In [ ]:
base_model_y_pred = base_model.predict(test_df.copy().drop(columns='y'))

### Variable Importance in Projection (VIP) Feature Selection

In [ ]:
import numpy as np

def compute_vip(pls, X):
    """
    Compute the Variable Importance in Projection (VIP) scores for each predictor (wavenumber).

    Parameters
    ----------
    pls : trained PLSRegression object
        A fitted scikit-learn PLSRegression model.
    X : ndarray or pandas dataframe, shape (n_samples, n_features)
        Training data on which the model was fit (or at least the same shape/features).

    Returns
    -------
    vip_scores : ndarray of shape (n_features,)
        VIP score for each feature (wavenumber).
    """
    # Extract the necessary parameters from the PLS model
    T = pls.x_scores_           # (n_samples, n_components)
    W = pls.x_weights_          # (n_features, n_components)
    Q = pls.y_loadings_.ravel() # (n_components,)

    # Number of features and components
    p, A = W.shape

    # Sum of squares explained by each component for Y
    # SS(t_h * q_h) ~ sum over all samples of (t_h * q_h)^2
    # We can do that by (T[:,h] * Q[h])^2 and summing
    SS = np.sum((T * Q)**2, axis=0)  # shape (n_components,)
    total_SS = np.sum(SS)           # total sum of squares

    # Initialize VIP array
    vip_scores = np.zeros(p)

    # For each component h
    for h in range(A):
        # Weight for feature j on component h = W[j,h]
        # Contribution factor = (T[:,h] * Q[h])^2
        # So we add the share of the SS explained by component h, scaled by W[j,h]^2
        vip_scores += (SS[h] * (W[:, h]**2)) / total_SS

    # Multiply by p and take the square root
    vip_scores = np.sqrt(p * vip_scores)
    return vip_scores

def sliding_window_smooth(values, window_size=20):
    """
    Smooth values with a simple moving average over `window_size`.
    """
    kernel = np.ones(window_size) / window_size
    return np.convolve(values, kernel, mode='same')

#### VIP Model Training

In [1]:
vip = compute_vip(pls, train_df.copy().drop(columns='y'))

window_size = 11

vip_smoothed = sliding_window_smooth(vip, window_size=window_size)

NameError: name 'compute_vip' is not defined

In [ ]:
import matplotlib.pyplot as plt

wavenumbers = np.linspace(4000, 650, num=train_df.shape[1]-1)

plt.figure(figsize=(10, 6))

plt.plot(wavenumbers, vip, label='VIP (raw)')
plt.plot(wavenumbers, vip_smoothed, label=f'VIP (smoothed, window={window_size})', linewidth=2)
plt.axhline(y=1.0, linestyle='dashed', color='black')
plt.axhline(y=0.8, linestyle='dotted', color='black')

plt.gca().invert_xaxis()  # Often MIR spectra are plotted from high to low wavenumber
plt.xlabel("Wavenumber (cm$^{-1}$)")
plt.ylabel("VIP Score")
plt.title("Variable Importance in Projection (VIP) Across Wavenumbers")
plt.legend()
plt.show()

#### VIP Model Evaluation

### Base and VIP Model Comparison